깊은 인공 신경망을 학습하다보면 역전파 과정에서 입력층으로 갈수록 기울기가 점차적으로 작아지는 현상이 발생할 수 있습니다. 입력층에 가까운 층들에서 가중치들이 업데이트가 제대로 되지 않으면 결국 최적의 모델을 찾을 수 없게 됩니다. 이를 **기울기 소실(Gradient Vanishing)** 이라고 합니다.

반대의 경우도 있습니다. 기울기가 점차 커지다가 비정상적으로 큰 값이 되면서 결국 발산되기도 합니다. 이를 **기울기 폭주(Gradient Exploding)** 이라고 하며, 다음에 나올 순환 신경망(Recurrent Neural Network, RNN)에서 발생할 수 있습니다.

## **1. ReLU와 ReLU의 변형들**

앞에서 배운 내용을 돌아봅시다. 시그모이드 함수를 사용하면 입력의 절대값이 클 경우, 시그모이드 함수의 출력값이 0 또는 1에 수렴하면서 기울기가 0에 가까워집니다. 그래서 역전파 과정에서 전파시킬 기울기가 점차 사라져서 입력층 방향으로 갌록 제대로 역전파가 되지 않는 기울기 소실 문제가 발생할 수 있습니다.

기울기 소실을 완화하는 가장 간단한 방법은 은닉층의 활성화 함수로 시그모이드나 tanh함수 대신에 ReLU나 ReLU의 변형 함수와 같은 Leaky ReLU를 사용하는 것입니다.

## **2. 가중치 초기화(Weight Initialization)**

같은 모델을 훈련시키더라도 가중치가 초기에 어떤 값을 가졌느냐에 따라서 모델의 훈련 결과가 달라지기도 합니다. 다시 말해 가중치 초기화를 적절히만 해도 기울기 소실 문제같은 문제를 완화시킬 수 있습니다.

### 1) 세이비어 초기화(Xavier Initialization)
2010년 세이비어 글로럿과 요슈아 벤지오는 가중치 초기화가 모델에 미치는 영향을 분석하여 새로운 초기화 방법을 제안했습니다. 이 초기화 방법을 제안한 세이비어의 이름을 따서 **세이비어 초기화** 또는 글로럿 초기화라고 합니다.

이 방법은 균등 분포(Uniform Distribution) 또는 정규 분포(Normal Distribution)로 초기화 할 때로 나뉘며, 이전 층의 뉴런의 개수를 $n_{in}$, 다음 층의 뉴런의 개수를 $n_{out}$이라고 해봅시다. 

세이비어와 글로럿의 논문에선 균등 분포를 사용하여 가중치를 초기화할 경우, 다음과 같은 균등 분포 범위를 사용하라고 합니다.

$$W \sim Uniform(-\sqrt{\frac{6}{n_{in} + n_{out}}}, +\sqrt{\frac{6}{n_{in} + n_{out}}})$$

다시 말해 $\sqrt{\frac{6}{n_{in} + n_{out}}}$을 $m$이라고 하면, $-m$과 $m$ 사이의 균등 분포를 의미합니다.

정규 분포로 초기화할 경우에는 평균이 0이고, 표준 편차가 $\sigma$가 다음을 만족해야 합니다.

$$\sigma = \sqrt{\frac{2}{n_{in} + n_{out}}}$$

세이비어 초기화는 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막습니다. 이러한 세이비어 초기화는 시그모이드 함수나 tanh 함수와 같이 S자 형태인 활성화 함수와 함께 사용할 경우 좋은 성능을 보이지만 ReLU와 함께 사용할 경우 성능이 좋지 않습니다. 

논문: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf

### 2) He 초기화(He initialization)

He 초기화는 ReLU 계열 함수와 사용할 때 효율적입니다. 세이비어 초기화와 유사하게 정규 분포와 균등 분포 두 가지 경우로 나뉩니다. 다만, He 초기화는 세이비어 초기화와 다르게 다음 층의 뉴런의 수를 반영하지 않습니다. 전과 같이 이전 층의 뉴런의 개수를 $n_{in}$이라고 해봅시다. 

$$W \sim Uniform(-\sqrt{\frac{6}{n_{in}}}, +\sqrt{\frac{6}{n_{in}}})$$

정규 분포로 초기화할 경우에는 표준 편차 $\sigma$가 다음을 만족하도록 합니다.

$$\sigma = \sqrt{\frac{2}{n_{in}}}$$

보편적으로 ReLU + He 초기화 방법을 많이 사용합니다.

논문: https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf

## **3. 배치 정규화(Batch Normalization)**

ReLU 계열의 함수와 He 초기화를 사용하는 것만으로 어느 정도 기울기 소실과 폭주를 완화시킬 수 있습니다. 그러나 이 두 방법만 사용하더라도 훈련 중 언제든지 다시 소실과 폭주가 발생할 수 있습니다. 이를 예방하기 위해 또 다른 방법인 배치 정규화를 사용합니다. **배치 정규화**는 인공 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만듭니다.

### 1) 내부 공변량 변화(Internal Covariate Shift)

배치 정규화를 이해하기 위해 먼저 내부 공변량 변화를 이해할 필요가 있습니다. 내부 공변량 변화는 학습 과정에서 **층 별로 입력 데이터 분포가 달라지는 현상**을 말합니다. 이전 층들의 학습에 의해 이전 층의 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력 데이터의 분포가 현재 층이 학습했던 시점의 분포와 차이가 발생합니다. 배치 정규화를 제안한 논문에서는 기울기 소실/폭주 등의 딥 러닝 모델의 불안전성이 층마다 입력의 분포가 달라지기 때문이라고 주장합니다. 

- 공변량 변화는 훈련 데이터의 분포와 테스트 데이터의 분포가 다른 경우를 의미합니다.
- 내부 공변량 변화는 신경망 층 사이에서 발생하는 입력 데이터의 분포 변화를 의미합니다.

### 2) 배치 정규화(Batch Normalization)
배치 정규화는 말그대로 한 번에 들어오는 배치 단위로 정규화하는 것을 말합니다. 배치 정규화는 각 층에서 활성화 함수를 통과하기 전에 수행됩니다. 

배치 정규화를 요약하면 다음과 같습니다.

입력에 대해 평균을 0으로 만들고, 정규화를 합니다. 그리고 정규화 된 데이터에 대해서 스케일과 시프트를 수행합니다. 이 때 두 개의 매개변수 $\gamma$와 $\beta$를 사용하는데, $\gamma$는 스케일을 위해 사용하고, $\beta$는 시프를 하는 것에 사용하며 다음 레이어에 일정한 범위의 값들만 전달되게 합니다.

배치 정규화의 수식은 다음과 같습니다. 아래에서 $BN$은 배치 정규화를 의미합니다.

Input: 미니 배치 $B={x^{(1)}, x^{(2)}, \cdots, x^{(m)}}$
Output: $y^{(i)} = BN_{\gamma, \beta}(x^{(i)})$

$$\mu \beta \leftarrow \frac{1}{m} \sum_{i=1}^m x^{(i)} \quad (미니 배치에 대한 평균)$$
$$\sigma_B^2 \leftarrow \frac{1}{m} \sum_{i=1}^m (x^{(i)} - \mu_B)^2 \quad (미니 배치에 대한 분산)$$
$$\hat{x}^{(i)} \leftarrow \frac{x^{(i)} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad (정규화)$$
$$y^{(i)} \leftarrow \gamma \hat{x^{(i)}} + \beta = BN_{\gamma, \beta}(x^{(i)}) \quad (스케일 조정과 시프트) $$

- $m$은 미니 배치에 있는 샘플의 수
- $\mu_B$는 미니 배치 $B$에 대한 평균
- $\sigma_B$는 미니 배치 $B$에 대한 표준 편차
- $\hat{X{(i)}}$은 평균이 0이고 정규화 된 입력 데이터
- $\epsilon$은 분모가 0이 되는 것을 막는 작은 수. 보편적으로 $10^{-5}$
- $\gamma$는 정규화 된 데이터에 대한 스케일 매개변수로 학습 대상
- $\beta$는 정규화 된 데이터에 대한 시프트 매개변수로 학습 대상
- $y^{(i)}$는 스케일과 시프트를 통해 조정한 $BN$의 최종 결과

배치 정규화는 학습 시, 배치 단위의 평균과 분산들을 차례대로 받아 이동 편균과 이동 분산을 저장해놓았다가 테스트 할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화를 합니다. 

- 배치 정규화를 사용하면 시그모이드 함수나 tanh 함수를 사용하더라도 기울기 소실 문제가 크게 개선됩니다.
- 가중치 초기화에 훨씬 덜 민감해집니다.
- 훨씬 큰 학습률을 사용할 수 있어 학습 속도를 개선시킵니다.
- 미니 배치마다 평균과 표준편차를 계산하므로 훈련 데이터에 일종의 잡음을 넣는 부수 효과로 과적합을 방지하는 효과도 냅니다. 하지만 부수적 효과이므로 드롭 아웃과 함께 사용하는 것이 좋습니다.
- 배치 정규화는 모델을 복잡하게 하며, 추가 계산을 하는 것이므로 테스트 데이터에 대한 예측 시에 실행 시간이 느려집니다. 그래서 서비스 속도를 고려하는 관점에서는 배치 정규화가 꼭 필요한지 고민이 필요합니다. 
- 배치 정규화의 효과는 굉장하지만 내부 공변량 변화때문은 아니라는 논문도 있습니다. (https://arxiv.org/pdf/1805.11604.pdf)



### 3) 배치 정규화의 한계

배치 정규화는 뛰어난 방법이지만 몇 가지 한계가 존재합니다. 

**1. 미니 배치 크기에 의존적이다.**

배치 정규화는 너무 작은 배치 크기에서는 잘 동작하지 않을 수 있습니다. 단적으로 배치 크기를 1로 하게되면 분산은 0이 됩니다. 작은 미니 배치에서는 배치 정규화의 효과가 극단적으로 작용되어 훈련에 악영향을 줄 수 있습니다. 배치 정규화를 적용할 때는 작은 미니 배치보다는 크기가 어느정도되는 미니 배치에서 하는 것이 좋습니다. 이처럼 배치 정규화는 배치 크기에 의존적인 면이 있습니다.

**2. RNN에 적용하기 어렵다.**

뒤에서 배울 RNN은 각 시점(time step)마다 다른 통계치를 가집니다. 이는 RNN에 배치 정규화를 적용하는 것을 어렵게 만듭니다. RNN에서 배치 정규화를 적용하기 위한 몇 가지 논문이 제시되어 있지만, 여기서는 이를 소개하는 대신 배치 크기에도 의존적이지 않으며, RNN에도 적용하는 것이 수월한 층 정규화(layer normalization)라는 방법을 소개하고자 합니다.

## **4. 층 정규화(Layer Normalization)**

층 정규화를 이해하기 앞서 배치 정규화를 시각화해보겠습니다. 다음은 $m$이 3이고 특성의 수가 4일 때 배치 정규화를 보여줍니다. 미니 배치란 동일한 특성(feature) 개수들을 가진 다수의 샘플들을 의미함을 상기합니다. 

<img src = "https://wikidocs.net/images/page/61375/%EB%B0%B0%EC%B9%98%EC%A0%95%EA%B7%9C%ED%99%94.PNG">

반면, 층 정규화는 다음과 같습니다.

<img src = "https://wikidocs.net/images/page/61375/%EC%B8%B5%EC%A0%95%EA%B7%9C%ED%99%94.PNG">

---

<참고 자료들>
가중치 초기화 참고 자료 :  
http://nlp.jbnu.ac.kr/AI2019/slides/ch05-1.pdf  
https://reniew.github.io/13/  
https://calcifer1009-dev.tistory.com/11  

배치 정규화 인터넷 강의(한국어 자막) :  
https://www.youtube.com/watch?v=tNIpEZLv_eg  
https://www.youtube.com/watch?v=em6dfRxYkYU  
https://www.youtube.com/watch?v=nUUqwaxLnWs  

배치 정규화 참고자료 :  
https://light-tree.tistory.com/139  
https://sacko.tistory.com/44?category=632408  
http://funmv2013.blogspot.com/2016/09/batch-normalization.html  
https://excelsior-cjh.tistory.com/178  
https://www.youtube.com/watch?v=HCEr5f-LfVE&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=17  